In [2]:
from utils.autocad import pipes_network_sytems
from utils.autocad import autocad_analyzing
from entities import entities
import pandas as pd
import os

pipes_table = pd.DataFrame()
pumps_table = pd.DataFrame()
channels_table = pd.DataFrame()

pipes_table, pumps_table, channels_table = autocad_analyzing.dwg_objects_sorting()
autocad_analyzing.is_pipe_conected(pipes_table,pumps_table)

with pd.ExcelWriter('data\\outputs\\acad-pipelines.xlsx') as writer:
        pipes_table.to_excel(writer,sheet_name='Pipes',index=False)
        pumps_table.to_excel(writer,sheet_name='Pumps',index=False)
        channels_table.to_excel(writer,sheet_name='Channels',index=False)    
os.startfile('data\\outputs\\acad-pipelines.xlsx')    

c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\entities\entities.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pipe_table['Id'][0]=0
c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\entities\entities.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pipe_table['Id'][0]=0
c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\entities\entities.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

the max flow rate is: 647.4865139418966 m^3/hr, but the desgin is for 1200.0 m^3/hr
Channel depth 1.2
the max flow rate is: 10101.841288290221 m^3/hr, but the desgin is for 1200.0 m^3/hr


In [3]:
junctions_list= list(pipes_table[pipes_table['end with'].str.contains(',', na=False)]['pipe #'])
junctions_list

['Pipe 3']

In [4]:
# only split, not ring

branches_tree = {}
for junction in junctions_list:
    branched_pipes = pipes_table['end with'][junction].split(', ')
    branches_tree[junction] = {}
    for i,pipe in enumerate(branched_pipes):
        # print(pipe)
        # for i,pipe in enumerate(pipes):
        #     print(pipe)
        branch_id = f'Branch {i+1}'
        new_pipe=pipe
        branch_pipes = []
        branch_consumption = 0
        while pipes_table['end with'][new_pipe] != '':
            branch_consumption += pipes_table['consumption'][new_pipe]
            branch_pipes.append(str(new_pipe))
            new_pipe= pipes_table['end with'][new_pipe]
        branch_consumption += pipes_table['consumption'][new_pipe]
        branch_pipes.append(str(new_pipe))
        print(branch_consumption)
      
        pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption
        branch_dict = {
            'pipes' : branch_pipes,
            'branch_consumption' : branch_consumption,
            'tested': False,
        }

        branches_tree[junction][branch_id] = branch_dict
        # print(branches)
        # pass
branches_tree
# pipes_table

800.0
800.0


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\498842354.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\498842354.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption


{'Pipe 3': {'Branch 1': {'pipes': ['Pipe 4', 'Pipe 5'],
   'branch_consumption': 800.0,
   'tested': False},
  'Branch 2': {'pipes': ['Pipe 6', 'Pipe 7'],
   'branch_consumption': 800.0,
   'tested': False}}}

In [5]:
def make_branches_df(pipes_table: pd.DataFrame):
    junctions_list= list(pipes_table[pipes_table['end with'].str.contains(',', na=False)]['pipe #'])
    branches_tree = {}
    for junction in junctions_list:
        branched_pipes = pipes_table['end with'][junction].split(', ')
        branches_tree[junction] = {}
        for i,pipe in enumerate(branched_pipes):
            branch_id = f'Branch {i+1}'
            new_pipe=pipe
            branch_pipes = []
            branch_consumption = 0
            while pipes_table['end with'][new_pipe] != '':
                branch_consumption += pipes_table['consumption'][new_pipe]
                branch_pipes.append(str(new_pipe))
                new_pipe= pipes_table['end with'][new_pipe]
            branch_consumption += pipes_table['consumption'][new_pipe]
            branch_pipes.append(str(new_pipe))
        
            pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption
            branch_dict = {
                'pipes' : branch_pipes,
                'branch_consumption' : branch_consumption,
                'tested': False,
            }

            branches_tree[junction][branch_id] = branch_dict
    print(branches_tree)
    branches_df = pd.DataFrame(branches_tree)
    print(branches_df)
    branches_df.set_index(branches_df[junction],inplace=True)
    
    return pd.DataFrame(branches_tree)

branches_df = make_branches_df(pipes_table)

{'Pipe 3': {'Branch 1': {'pipes': ['Pipe 4', 'Pipe 5'], 'branch_consumption': 800.0, 'tested': False}, 'Branch 2': {'pipes': ['Pipe 6', 'Pipe 7'], 'branch_consumption': 800.0, 'tested': False}}}
                                                     Pipe 3
Branch 1  {'pipes': ['Pipe 4', 'Pipe 5'], 'branch_consum...
Branch 2  {'pipes': ['Pipe 6', 'Pipe 7'], 'branch_consum...


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\2307108510.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\2307108510.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption


In [6]:
branches_df

,Pipe 3
Branch 1,"{'pipes': ['Pipe 4', 'Pipe 5'], 'branch_consum..."
Branch 2,"{'pipes': ['Pipe 6', 'Pipe 7'], 'branch_consum..."


In [9]:
def next_branches_not_tested(branches_df:pd.DataFrame)->str:
    not_tested_branches = branches_df[~branches_df['Tested']]
    if not not_tested_branches.empty:
        next_pipe = not_tested_branches.iloc[0]['Pipes'][0]
        branch_num = not_tested_branches.iloc[0]['Branch']
        # branches_df.at[not_tested_branches.index[0], 'Tested'] = True
        branches_df[(branches_df['Pipe']==not_tested_branches.index[0]) & (branches_df['Branch']==branch_num)]['Tested'] = True
        return next_pipe
    else:
        return None

def reset_branches_tested_values(branches_df):
    branches_df['Tested'] = False  

def make_branches_df(pipes_table: pd.DataFrame):
    junctions_list = list(pipes_table[pipes_table['end with'].str.contains(',', na=False)]['pipe #'])
    data = []

    for junction in junctions_list:
        branched_pipes = pipes_table['end with'][junction].split(', ')
        for i, pipe in enumerate(branched_pipes):
            branch_id = f'Branch {i+1}'
            new_pipe = pipe
            branch_pipes = []
            branch_consumption = 0
            while pipes_table['end with'][new_pipe] != '':
                branch_consumption += pipes_table['consumption'][new_pipe]
                branch_pipes.append(str(new_pipe))
                new_pipe = pipes_table['end with'][new_pipe]
            branch_consumption += pipes_table['consumption'][new_pipe]
            branch_pipes.append(str(new_pipe))

            pipes_table['flow'][branch_pipes[0]] = branch_consumption
            data.append((junction, branch_id, branch_pipes, branch_consumption, False))

    branches_df = pd.DataFrame(data, columns=['Pipe', 'Branch', 'Pipes', 'Branch Consumption', 'Tested'])
    branches_df.set_index(branches_df['Pipe'],inplace=True)
    print(branches_df)
    return branches_df



branches_df = make_branches_df(pipes_table)

pump_number = pumps_table['pump #'][0]
pump_flow = pipes_table['consumption'].sum()  
pumps_table['flow'][pump_number] = pump_flow
pump1 = entities.Pump(rated_flow=pump_flow)

previous_pipe_number = 0

for pipe in range(0,len(pipes_table.index)):

    # branch_index = 1
    if pipe == 0:
        new_pipe_number = pipes_table[pipes_table['start with'] == pump_number]['end with'].index[0]
        flow_rate = pump_flow
    elif (previous_pipe_number in branches_df.index) or (pipes_table['end with'][previous_pipe_number] == ""):
        new_pipe_number = next_branches_not_tested(branches_df)
        print(new_pipe_number)
        flow_rate = pipes_table['flow'][new_pipe_number]

    else:
        new_pipe_number = pipes_table['end with'][previous_pipe_number]
        flow_in = float(pipes_table['flow'][previous_pipe_number])
        consumption = float(pipes_table['consumption'][previous_pipe_number])
        flow_rate = flow_in - consumption
        
    pipetype = pipes_table['pipe type'][new_pipe_number]
    nominal_dia = pipes_table['nominal dia'][new_pipe_number]
    inside_dia = pipes_table['id (mm)'][new_pipe_number]
    length = pipes_table['length (m)'][new_pipe_number]
    static_head = pipes_table['static head (endZ-startZ)'][new_pipe_number]

    pipe1 = entities.Pipe(pipetype=pipetype,nominal_dia=nominal_dia,inside_dia=inside_dia,length=length,static_head=static_head,flow_rate=flow_rate)

    velocity = pipe1.velocity()
    head_loss = pipe1.major_head_loss()
    total_headloss = pipe1.total_head_loss()

    pipes_table['flow'][new_pipe_number] = flow_rate 
    pipes_table['velocity'][new_pipe_number] = velocity
    pipes_table['head loss'][new_pipe_number] = head_loss
    pipes_table['total head loss'][new_pipe_number] = total_headloss

    previous_pipe_number = new_pipe_number
reset_branches_tested_values(branches_df)

          Pipe    Branch             Pipes  Branch Consumption  Tested
Pipe                                                                  
Pipe 3  Pipe 3  Branch 1  [Pipe 4, Pipe 5]               800.0   False
Pipe 3  Pipe 3  Branch 2  [Pipe 6, Pipe 7]               800.0   False


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\3169552628.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][branch_pipes[0]] = branch_consumption
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\3169552628.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][branch_pipes[0]] = branch_consumption
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_24284\3169552628.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

KeyboardInterrupt: 

In [8]:
pipes_table

,pipe #,start,end,pipe type,nominal dia,id (mm),length (m),static head (endZ-startZ),flow,consumption,minimum pressure required,velocity,head loss,total head loss,Pressure at end of pipe,start with,end with,costs per meter,total costs
pipe #,,,,,,,,,,,,,,,,,,,
Pipe 1,Pipe 1,"(536.08593755699, 1927.8727861776483, -5.0)","(1031.5975623507745, 1994.7176041834116, 10.0)",PE100-16,800.0,654.8,500.224949,15.000000,2400.0,500.0,0.0,1.97971,2.417168,17.617008,,Pump 1,Pipe 2,3684.020571,1.842839e+06
Pipe 2,Pipe 2,"(1031.5975623507745, 1994.7176041834116, 10.0)","(275.5339499512405, 3290.2359614463903, 3.0)",Steel,20.0,484.0,1500.016333,-7.000000,1900.0,0.0,0.0,2.868603,20.492279,13.911863,,Pipe 1,Pipe 3,1140.000000,1.710019e+06
Pipe 3,Pipe 3,"(275.5339499512405, 3290.2359614463903, 3.0)","(100.02140612523948, 2987.4235657956738, 5.0)",PE100-10,500.0,440.6,350.005714,2.000000,1900.0,300.0,70.0,3.461563,7.555612,10.166586,,Pipe 2,"Pipe 4, Pipe 6",1200.000000,4.200069e+05
Pipe 4,Pipe 4,"(100.02140612523948, 2987.4235657956738, 5.0)","(-710.8799755422069, 3248.8136112868197, 10.0)",PE100-12.5,315.0,268.6,852.003995,5.000000,800.0,500.0,20.0,3.921801,41.272193,47.056433,,Pipe 3,Pipe 5,640.000000,5.452826e+05
Pipe 5,Pipe 5,"(-710.8799755422069, 3248.8136112868197, 10.0)","(-963.7318520066838, 2783.554092127968, 0.0)",Steel,10.0,253.0,529.622971,-10.000000,300.0,300.0,0.0,1.657631,5.582619,-4.277276,,Pipe 4,,520.000000,2.754039e+05
Pipe 6,Pipe 6,"(100.02140612523948, 2987.4235657956738, 5.0)","(-224.57043511596868, 2427.404145460851, 8.698...",Steel,10.0,253.0,647.298459,3.698788,800.0,300.0,0.0,,,,,Pipe 3,Pipe 7,520.000000,3.365952e+05
Pipe 7,Pipe 7,"(-224.57043511596868, 2427.404145460851, 8.698...","(-749.0632077821901, 2530.194204976131, 10.0)",PE100-12.5,315.0,268.6,534.471850,1.301212,,500.0,20.0,,,,,Pipe 6,,640.000000,3.420620e+05
